On 1-Dec-2020, the S&P500 (SPX) index value was 3662.45, while the SPDR
S&P500 Exchange Traded Fund (SPY) stock price was 366.02. The call and
put option prices (bid & offer) over 3 maturities are provided in the
spreadsheet:
• SPX options.csv
• SPY options.csv
The discount rate on this day is in the file: zero rates 20201201.csv.
Calibrate the following models to match the option prices:
1 Displaced-diffusion model
2 SABR model (fix β = 0.7)
Plot the fitted implied volatility smile against the market data.
Report the model parameters:
1 σ, β
2 α, ρ, ν
And discuss how does change β in the displaced-diffusion model and ρ, ν in the
SABR model affect the shape of the implied volatility

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt

In [4]:
df_spx = pd.read_csv("SPX_options.csv")
df_spy = pd.read_csv("SPY_options.csv")
df_rates = pd.read_csv("zero_rates_20201201.csv")


In [6]:
df_spy.describe()

,date,exdate,strike_price,best_bid,best_offer
count,1400.0,1.400000e+03,1400.000000,1400.000000,1400.000000
mean,20201201.0,2.020690e+07,304117.142857,45.799986,46.013850
std,0.0,4.302003e+03,95474.829262,66.401394,66.576624
min,20201201.0,2.020122e+07,25000.000000,0.000000,0.010000
25%,20201201.0,2.020122e+07,242000.000000,0.300000,0.317500
50%,20201201.0,2.021012e+07,309000.000000,10.060000,10.105000
75%,20201201.0,2.021012e+07,367250.000000,74.575000,75.280000
max,20201201.0,2.021022e+07,555000.000000,340.760000,341.230000


In [2]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def black_price(F0, K, T, r, sig)->tuple:
    """
    Calculate the Black model prices of European call and put options.

    Parameters:
    ----------
    F0 : float
        Current futures price.
    K : float
        Strike price of the option.
    T : float
        Time to expiration in years.
    r : float
        Risk-free interest rate (as a decimal).
    sig : float
        Volatility of the underlying asset (annualized).

    Returns:
    -------
    tuple
        A tuple containing the call option price and the put option price (call_price, put_price).

    Example:
    -------
    >>> C, P = black_price(50, 50, 0.5, 0.05, 0.2)
    >>> print( black_price(50, 50, 0.5, 0.05, 0.2) )
    (55.60808486946355, 0.0)
    """

    d = (math.log(F0/K) +.5*(sig**2)*T) / \
          (sig*T**.5)
    
    V_c = math.exp(-r*T) * ( F0*norm.cdf( d) - K *norm.cdf( d) )
    V_p = math.exp(-r*T) * ( K *norm.cdf(-d) - F0*norm.cdf(-d) )
    
    return V_c, V_p

def displaced_diffusion_price(F0, K, T, r, sig, beta)->tuple:
    """
    Calculate the price of European call and put options using the Displaced Diffusion model.

    The Displaced Diffusion model modifies the Black model by applying a displacement
    to the futures price and strike price, which helps in better fitting the observed 
    market prices of options.

    Parameters:
    ----------
    F0 : float
        Current futures price.
    K : float
        Strike price of the option.
    T : float
        Time to expiration in years.
    r : float
        Risk-free interest rate (as a decimal).
    sig : float
        Volatility of the underlying asset (annualized).
    beta : float
        Displacement factor, where 0 < beta < 1. Affects the adjustment of the futures
        price and strike price.

    Returns:
    -------
    tuple
        A tuple containing the call option price and the put option price 
        (call_price, put_price) calculated using the Black model with the adjusted parameters.

    Example:
    -------
    >>> C, P = displaced_diffusion_price(50, 50, 0.5, 0.05, 0.5, 0.4)
    >>> print(C,P)
    128.78625759180068, -6.929993731065541e-15
    """
    F0_dd   = F0 / beta
    K_dd    = K + (1-beta)/beta * F0
    sig_dd  = sig * beta

    return black_price(F0_dd, K_dd, T, r, sig_dd)

In [3]:
from scipy.optimize import least_squares

beta = 0.7

def ddcalibration(x, strikes, vols, F, T):
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR(F, strikes[i], T,
                           x[0], beta, x[1], x[2]))**2

    return err

def sabrcalibration(x, strikes, vols, F, T):
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR(F, strikes[i], T,                # change to Displaced-Diffusion
                           x[0], beta, x[1], x[2]))**2

    return err


def impliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalCall(S, K, r, x, T),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol


def BlackScholesLognormalCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesLognormalPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)


#####
# Here, load DataFrame with strike and implied volatility information into "df"
#####
df = pd.read_csv('SPX_options.csv')
df['mid'] = 0.5*(df['best_bid'] + df['best_offer'])
df['strike'] = df['strike_price']*0.001
df['payoff'] = df['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
exdate = sorted(df['exdate'].unique())[0]
df = df[df['exdate'] == exdate]
days_to_expiry = (pd.Timestamp(str(exdate)) - pd.Timestamp('2020-12-01')).days
T = days_to_expiry/365
S = 3662.45
r = 0.14/100.0
F = S*np.exp(r*T)

df['vols'] = df.apply(lambda x: impliedVolatility(S,
                                                  x['strike'],
                                                  r,
                                                  x['mid'],
                                                  T,
                                                  x['payoff']),
                      axis=1)
df.dropna(inplace=True)
call_df = df[df['payoff'] == 'call']
put_df = df[df['payoff'] == 'put']
strikes = put_df['strike'].values
impliedvols = []
for K in strikes:    
    if K > S:
        impliedvols.append(call_df[call_df['strike'] == K]['vols'].values[0])
    else:
        impliedvols.append(put_df[put_df['strike'] == K]['vols'].values[0])

# populate "df" with the dataframe containing strikes and market implied volatilities
df = pd.DataFrame({'strike': strikes, 'impliedvol': impliedvols})

initialGuess = [0.02, 0.2, 0.1]
res = least_squares(lambda x: sabrcalibration(x,
                                              df['strike'],
                                              df['impliedvol'],
                                              F,
                                              T),
                    initialGuess)
alpha = res.x[0]
rho = res.x[1]
nu = res.x[2]

print('Calibrated SABR model parameters: alpha = %.3f, beta = %.1f, rho = %.3f, nu = %.3f' % (alpha, beta, rho, nu))

sabrvols = []
for K in strikes:
    sabrvols.append(SABR(F, K, T, alpha, beta, rho, nu))

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_94443/873058113.py:12: RuntimeWarning: invalid value encountered in double_scalars
  zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_94443/873058113.py:12: RuntimeWarning: invalid value encountered in log
  zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))


Calibrated SABR model parameters: alpha = 1.212, beta = 0.7, rho = -0.301, nu = 5.460


In [ ]:
# populate "df" with the dataframe containing strikes and market implied volatilities
df = pd.DataFrame({'strike': strikes, 'impliedvol': impliedvols})

initialGuess = [0.02, 0.2, 0.1]
res = least_squares(lambda x: sabrcalibration(x,
                                              df['strike'],
                                              df['impliedvol'],
                                              F,
                                              T),
                    initialGuess)
alpha = res.x[0]
rho = res.x[1]
nu = res.x[2]

print('Calibrated SABR model parameters: alpha = %.3f, beta = %.1f, rho = %.3f, nu = %.3f' % (alpha, beta, rho, nu))

sabrvols = []
for K in strikes:
    sabrvols.append(SABR(F, K, T, alpha, beta, rho, nu))